In [4]:
!pip install nibabel

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 87.1 MB/s eta 0:00:00:00:01

[notice] A new release of pip is available: 23.0.1 -> 23.2.1
[notice] To update, run: python3 -m pip install --upgrade pip


In [2]:
import pandas as pd
import os
import numpy as np
import nibabel as nib
import matplotlib.pyplot as plt
import cv2
import glob

os.getcwd()
os.chdir('/home/ubuntu/4catalyzer-hackathon/')

In [27]:
import matplotlib.pyplot as plt

def convert_nifti_to_png_axial(input_dir, output_dir):
    # Create the output directory if it doesn't exist
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    for root, _, files in os.walk(input_dir):
        for file in files:
            _, file_extension = os.path.splitext(file)
            if 'dwi.nii.gz' in file:
                file_path = os.path.join(root, file)

                # Extract the case number from the lesion file name
                case_number = os.path.basename(file).split("_")[0][4:]

                # Find the corresponding MRI file
                mri_file = os.path.join(input_dir, f"sub-{case_number}", "ses-01", "dwi", f"sub-{case_number}_ses-01_dir-unflipped_dwi.nii.gz")
                mri_file = glob.glob(mri_file)
                
                # Check if an MRI file is found for the current case
                if len(mri_file) == 1:
                    mri_file = mri_file[0]
                else:
                    print(f"No MRI file found for case {case_number}. Skipping...")
                    continue
                
                # Load the DWI NIfTI data
                dwi_img = nib.load(mri_file)
                dwi_data = dwi_img.get_fdata()

                # Check the dimensionality of the data
#                 print(dwi_data.shape)
#                 if len(dwi_data.shape) != 3:
#                     print(f"Skipping {mri_file} due to non-axial slice orientation.")
#                     continue


                # Create the DWI axial slice images
                for i in range(dwi_data.shape[2]):
#                     for j in range(dwi_data.shape[3]):
                    dwi_slice = dwi_data[:, :, i, 17]

                    # Normalize the DWI slice to have values between 0 and 1 for better visualization
                    dwi_slice_normalized = (dwi_slice - dwi_slice.min()) / (dwi_slice.max() - dwi_slice.min())

                    # Create and save the PNG image
                    output_file = os.path.join(output_dir, f"sub-{case_number}", f"case{case_number}_dwi_slice_{i}.png")
                    os.makedirs(os.path.dirname(output_file), exist_ok=True)  # Create case sub-directory if it doesn't exist
                    plt.imshow(dwi_slice_normalized, cmap='gray')
                    plt.axis('off')
                    plt.savefig(output_file, bbox_inches='tight', pad_inches=0, dpi=100)
                    plt.close()  # Clear the current plot after saving each image

                print(f"Processed {mri_file} and saved the DWI axial slices as PNG to {os.path.join(output_dir, f'sub-{case_number}')}.")

In [28]:
input_directory = "./data/ds004215-download"
output_directory = "./data/healthy_images"
convert_nifti_to_png_axial(input_directory, output_directory)

Processed ./data/ds004215-download/sub-ON78603/ses-01/dwi/sub-ON78603_ses-01_dir-unflipped_dwi.nii.gz and saved the DWI axial slices as PNG to ./data/healthy_images/sub-ON78603.
Processed ./data/ds004215-download/sub-ON78603/ses-01/dwi/sub-ON78603_ses-01_dir-unflipped_dwi.nii.gz and saved the DWI axial slices as PNG to ./data/healthy_images/sub-ON78603.
Processed ./data/ds004215-download/sub-ON65553/ses-01/dwi/sub-ON65553_ses-01_dir-unflipped_dwi.nii.gz and saved the DWI axial slices as PNG to ./data/healthy_images/sub-ON65553.
Processed ./data/ds004215-download/sub-ON65553/ses-01/dwi/sub-ON65553_ses-01_dir-unflipped_dwi.nii.gz and saved the DWI axial slices as PNG to ./data/healthy_images/sub-ON65553.
Processed ./data/ds004215-download/sub-ON62592/ses-01/dwi/sub-ON62592_ses-01_dir-unflipped_dwi.nii.gz and saved the DWI axial slices as PNG to ./data/healthy_images/sub-ON62592.
Processed ./data/ds004215-download/sub-ON62592/ses-01/dwi/sub-ON62592_ses-01_dir-unflipped_dwi.nii.gz and save

Processed ./data/ds004215-download/sub-ON66557/ses-01/dwi/sub-ON66557_ses-01_dir-unflipped_dwi.nii.gz and saved the DWI axial slices as PNG to ./data/healthy_images/sub-ON66557.
Processed ./data/ds004215-download/sub-ON82386/ses-01/dwi/sub-ON82386_ses-01_dir-unflipped_dwi.nii.gz and saved the DWI axial slices as PNG to ./data/healthy_images/sub-ON82386.
Processed ./data/ds004215-download/sub-ON82386/ses-01/dwi/sub-ON82386_ses-01_dir-unflipped_dwi.nii.gz and saved the DWI axial slices as PNG to ./data/healthy_images/sub-ON82386.
Processed ./data/ds004215-download/sub-ON42107/ses-01/dwi/sub-ON42107_ses-01_dir-unflipped_dwi.nii.gz and saved the DWI axial slices as PNG to ./data/healthy_images/sub-ON42107.
Processed ./data/ds004215-download/sub-ON42107/ses-01/dwi/sub-ON42107_ses-01_dir-unflipped_dwi.nii.gz and saved the DWI axial slices as PNG to ./data/healthy_images/sub-ON42107.
Processed ./data/ds004215-download/sub-ON67435/ses-01/dwi/sub-ON67435_ses-01_dir-unflipped_dwi.nii.gz and save

In [5]:
def remove_black_png_images(directory):
    # Get a list of all files in the directory
    file_list = os.listdir(directory)

    # Process each file in the directory
    for file_name in file_list:
        file_path = os.path.join(directory, file_name)
        if os.path.isfile(file_path) and file_name.endswith(".png"):
            try:
                # Read the PNG image using cv2
                image = cv2.imread(file_path, cv2.IMREAD_GRAYSCALE)

                # Check if all pixel values are 0 (black)
                if not image.any():
                    # Delete the file
                    os.remove(file_path)
                    print(f"Deleted {file_name} (completely black).")
            except Exception as e:
                print(f"Error processing {file_name}: {e}")

In [6]:
remove_black_png_images('./data/healthy-images')

Deleted sub-ON65553_ses-01_acq-Sagittal3d_rec-SCIC_FLAIR.nii_axial_slice_254.png (completely black).
Deleted sub-ON65553_ses-01_acq-Sagittal3d_rec-SCIC_FLAIR.nii_axial_slice_255.png (completely black).
Deleted sub-ON65553_ses-01_acq-Sagittal3d_rec-SCIC_FLAIR.nii_axial_slice_252.png (completely black).
Deleted sub-ON65553_ses-01_acq-Sagittal3d_rec-SCIC_FLAIR.nii_axial_slice_251.png (completely black).
Deleted sub-ON62003_ses-01_acq-3dCUBE_rec-SCIC_FLAIR.nii_axial_slice_254.png (completely black).
Deleted sub-ON62003_ses-01_acq-3dCUBE_rec-SCIC_FLAIR.nii_axial_slice_252.png (completely black).
Deleted sub-ON65553_ses-01_acq-Sagittal3d_rec-SCIC_FLAIR.nii_axial_slice_253.png (completely black).
Deleted sub-ON62003_ses-01_acq-3dCUBE_rec-SCIC_FLAIR.nii_axial_slice_253.png (completely black).
Deleted sub-ON62003_ses-01_acq-3dCUBE_rec-SCIC_FLAIR.nii_axial_slice_255.png (completely black).
Deleted sub-ON62003_ses-01_acq-3dCUBE_rec-SCIC_FLAIR.nii_axial_slice_250.png (completely black).
Deleted su

In [7]:
remove_black_png_images('./data/lesion-images')

Deleted sub-strokecase0223_ses-0001_FLAIR.nii_axial_slice_14.png (completely black).
Deleted sub-strokecase0078_ses-0001_FLAIR.nii_axial_slice_43.png (completely black).
Deleted sub-strokecase0217_ses-0001_FLAIR.nii_axial_slice_53.png (completely black).
Deleted sub-strokecase0047_ses-0001_FLAIR.nii_axial_slice_327.png (completely black).
Deleted sub-strokecase0022_ses-0001_FLAIR.nii_axial_slice_14.png (completely black).
Deleted sub-strokecase0173_ses-0001_FLAIR.nii_axial_slice_34.png (completely black).
Deleted sub-strokecase0247_ses-0001_FLAIR.nii_axial_slice_39.png (completely black).
Deleted sub-strokecase0204_ses-0001_FLAIR.nii_axial_slice_323.png (completely black).
Deleted sub-strokecase0158_ses-0001_FLAIR.nii_axial_slice_0.png (completely black).
Deleted sub-strokecase0079_ses-0001_FLAIR.nii_axial_slice_1.png (completely black).
Deleted sub-strokecase0026_ses-0001_FLAIR.nii_axial_slice_13.png (completely black).
Deleted sub-strokecase0064_ses-0001_FLAIR.nii_axial_slice_119.png

Deleted sub-strokecase0235_ses-0001_FLAIR.nii_axial_slice_55.png (completely black).
Deleted sub-strokecase0065_ses-0001_FLAIR.nii_axial_slice_51.png (completely black).
Deleted sub-strokecase0113_ses-0001_FLAIR.nii_axial_slice_78.png (completely black).
Deleted sub-strokecase0003_ses-0001_FLAIR.nii_axial_slice_46.png (completely black).
Deleted sub-strokecase0059_ses-0001_FLAIR.nii_axial_slice_97.png (completely black).
Deleted sub-strokecase0044_ses-0001_FLAIR.nii_axial_slice_4.png (completely black).
Deleted sub-strokecase0023_ses-0001_FLAIR.nii_axial_slice_90.png (completely black).
Deleted sub-strokecase0119_ses-0001_FLAIR.nii_axial_slice_330.png (completely black).
Deleted sub-strokecase0126_ses-0001_FLAIR.nii_axial_slice_1.png (completely black).
Deleted sub-strokecase0174_ses-0001_FLAIR.nii_axial_slice_83.png (completely black).
Deleted sub-strokecase0163_ses-0001_FLAIR.nii_axial_slice_79.png (completely black).
Deleted sub-strokecase0244_ses-0001_FLAIR.nii_axial_slice_68.png (

Deleted sub-strokecase0181_ses-0001_FLAIR.nii_axial_slice_328.png (completely black).
Deleted sub-strokecase0112_ses-0001_FLAIR.nii_axial_slice_92.png (completely black).
Deleted sub-strokecase0069_ses-0001_FLAIR.nii_axial_slice_99.png (completely black).
Deleted sub-strokecase0135_ses-0001_FLAIR.nii_axial_slice_14.png (completely black).
Deleted sub-strokecase0114_ses-0001_FLAIR.nii_axial_slice_329.png (completely black).
Deleted sub-strokecase0231_ses-0001_FLAIR.nii_axial_slice_301.png (completely black).
Deleted sub-strokecase0136_ses-0001_FLAIR.nii_axial_slice_344.png (completely black).
Deleted sub-strokecase0172_ses-0001_FLAIR.nii_axial_slice_86.png (completely black).
Deleted sub-strokecase0086_ses-0001_FLAIR.nii_axial_slice_4.png (completely black).
Deleted sub-strokecase0075_ses-0001_FLAIR.nii_axial_slice_28.png (completely black).
Deleted sub-strokecase0103_ses-0001_FLAIR.nii_axial_slice_40.png (completely black).
Deleted sub-strokecase0163_ses-0001_FLAIR.nii_axial_slice_138.

Deleted sub-strokecase0002_ses-0001_FLAIR.nii_axial_slice_117.png (completely black).
Deleted sub-strokecase0044_ses-0001_FLAIR.nii_axial_slice_345.png (completely black).
Deleted sub-strokecase0066_ses-0001_FLAIR.nii_axial_slice_311.png (completely black).
Deleted sub-strokecase0116_ses-0001_FLAIR.nii_axial_slice_77.png (completely black).
Deleted sub-strokecase0120_ses-0001_FLAIR.nii_axial_slice_78.png (completely black).
Deleted sub-strokecase0063_ses-0001_FLAIR.nii_axial_slice_100.png (completely black).
Deleted sub-strokecase0029_ses-0001_FLAIR.nii_axial_slice_78.png (completely black).
Deleted sub-strokecase0108_ses-0001_FLAIR.nii_axial_slice_108.png (completely black).
Deleted sub-strokecase0073_ses-0001_FLAIR.nii_axial_slice_18.png (completely black).
Deleted sub-strokecase0156_ses-0001_FLAIR.nii_axial_slice_66.png (completely black).
Deleted sub-strokecase0107_ses-0001_FLAIR.nii_axial_slice_307.png (completely black).
Deleted sub-strokecase0180_ses-0001_FLAIR.nii_axial_slice_7

Deleted sub-strokecase0173_ses-0001_FLAIR.nii_axial_slice_58.png (completely black).
Deleted sub-strokecase0213_ses-0001_FLAIR.nii_axial_slice_36.png (completely black).
Deleted sub-strokecase0085_ses-0001_FLAIR.nii_axial_slice_45.png (completely black).
Deleted sub-strokecase0066_ses-0001_FLAIR.nii_axial_slice_58.png (completely black).
Deleted sub-strokecase0107_ses-0001_FLAIR.nii_axial_slice_8.png (completely black).
Deleted sub-strokecase0089_ses-0001_FLAIR.nii_axial_slice_71.png (completely black).
Deleted sub-strokecase0039_ses-0001_FLAIR.nii_axial_slice_63.png (completely black).
Deleted sub-strokecase0016_ses-0001_FLAIR.nii_axial_slice_347.png (completely black).
Deleted sub-strokecase0113_ses-0001_FLAIR.nii_axial_slice_91.png (completely black).
Deleted sub-strokecase0124_ses-0001_FLAIR.nii_axial_slice_27.png (completely black).
Deleted sub-strokecase0090_ses-0001_FLAIR.nii_axial_slice_104.png (completely black).
Deleted sub-strokecase0237_ses-0001_FLAIR.nii_axial_slice_329.pn

...

Deleted sub-strokecase0055_ses-0001_FLAIR.nii_axial_slice_19.png (completely black).
Deleted sub-strokecase0008_ses-0001_FLAIR.nii_axial_slice_50.png (completely black).
Deleted sub-strokecase0126_ses-0001_FLAIR.nii_axial_slice_66.png (completely black).
Deleted sub-strokecase0133_ses-0001_FLAIR.nii_axial_slice_92.png (completely black).
Deleted sub-strokecase0001_ses-0001_FLAIR.nii_axial_slice_323.png (completely black).
Deleted sub-strokecase0076_ses-0001_FLAIR.nii_axial_slice_3.png (completely black).
Deleted sub-strokecase0117_ses-0001_FLAIR.nii_axial_slice_302.png (completely black).
Deleted sub-strokecase0059_ses-0001_FLAIR.nii_axial_slice_110.png (completely black).
Deleted sub-strokecase0179_ses-0001_FLAIR.nii_axial_slice_101.png (completely black).
Deleted sub-strokecase0020_ses-0001_FLAIR.nii_axial_slice_31.png (completely black).
Deleted sub-strokecase0134_ses-0001_FLAIR.nii_axial_slice_340.png (completely black).
Deleted sub-strokecase0181_ses-0001_FLAIR.nii_axial_slice_77.

Deleted sub-strokecase0061_ses-0001_FLAIR.nii_axial_slice_78.png (completely black).
Deleted sub-strokecase0169_ses-0001_FLAIR.nii_axial_slice_8.png (completely black).
Deleted sub-strokecase0150_ses-0001_FLAIR.nii_axial_slice_19.png (completely black).
Deleted sub-strokecase0024_ses-0001_FLAIR.nii_axial_slice_42.png (completely black).
Deleted sub-strokecase0203_ses-0001_FLAIR.nii_axial_slice_88.png (completely black).
Deleted sub-strokecase0079_ses-0001_FLAIR.nii_axial_slice_317.png (completely black).
Deleted sub-strokecase0088_ses-0001_FLAIR.nii_axial_slice_46.png (completely black).
Deleted sub-strokecase0158_ses-0001_FLAIR.nii_axial_slice_4.png (completely black).
Deleted sub-strokecase0061_ses-0001_FLAIR.nii_axial_slice_304.png (completely black).
Deleted sub-strokecase0003_ses-0001_FLAIR.nii_axial_slice_327.png (completely black).
Deleted sub-strokecase0220_ses-0001_FLAIR.nii_axial_slice_322.png (completely black).
Deleted sub-strokecase0119_ses-0001_FLAIR.nii_axial_slice_52.pn

Deleted sub-strokecase0063_ses-0001_FLAIR.nii_axial_slice_345.png (completely black).
Deleted sub-strokecase0091_ses-0001_FLAIR.nii_axial_slice_17.png (completely black).
Deleted sub-strokecase0120_ses-0001_FLAIR.nii_axial_slice_345.png (completely black).
Deleted sub-strokecase0022_ses-0001_FLAIR.nii_axial_slice_121.png (completely black).
Deleted sub-strokecase0149_ses-0001_FLAIR.nii_axial_slice_15.png (completely black).
Deleted sub-strokecase0233_ses-0001_FLAIR.nii_axial_slice_296.png (completely black).
Deleted sub-strokecase0067_ses-0001_FLAIR.nii_axial_slice_99.png (completely black).
Deleted sub-strokecase0063_ses-0001_FLAIR.nii_axial_slice_80.png (completely black).
Deleted sub-strokecase0132_ses-0001_FLAIR.nii_axial_slice_44.png (completely black).
Deleted sub-strokecase0091_ses-0001_FLAIR.nii_axial_slice_215.png (completely black).
Deleted sub-strokecase0217_ses-0001_FLAIR.nii_axial_slice_72.png (completely black).
Deleted sub-strokecase0032_ses-0001_FLAIR.nii_axial_slice_95

Deleted sub-strokecase0181_ses-0001_FLAIR.nii_axial_slice_25.png (completely black).
Deleted sub-strokecase0158_ses-0001_FLAIR.nii_axial_slice_311.png (completely black).
Deleted sub-strokecase0098_ses-0001_FLAIR.nii_axial_slice_69.png (completely black).
Deleted sub-strokecase0010_ses-0001_FLAIR.nii_axial_slice_72.png (completely black).
Deleted sub-strokecase0100_ses-0001_FLAIR.nii_axial_slice_49.png (completely black).
Deleted sub-strokecase0172_ses-0001_FLAIR.nii_axial_slice_334.png (completely black).
Deleted sub-strokecase0054_ses-0001_FLAIR.nii_axial_slice_317.png (completely black).
Deleted sub-strokecase0126_ses-0001_FLAIR.nii_axial_slice_90.png (completely black).
Deleted sub-strokecase0139_ses-0001_FLAIR.nii_axial_slice_95.png (completely black).
Deleted sub-strokecase0139_ses-0001_FLAIR.nii_axial_slice_327.png (completely black).
Deleted sub-strokecase0078_ses-0001_FLAIR.nii_axial_slice_341.png (completely black).
Deleted sub-strokecase0116_ses-0001_FLAIR.nii_axial_slice_73

Deleted sub-strokecase0053_ses-0001_FLAIR.nii_axial_slice_14.png (completely black).
Deleted sub-strokecase0061_ses-0001_FLAIR.nii_axial_slice_344.png (completely black).
Deleted sub-strokecase0010_ses-0001_FLAIR.nii_axial_slice_10.png (completely black).
Deleted sub-strokecase0004_ses-0001_FLAIR.nii_axial_slice_114.png (completely black).
Deleted sub-strokecase0085_ses-0001_FLAIR.nii_axial_slice_107.png (completely black).
Deleted sub-strokecase0213_ses-0001_FLAIR.nii_axial_slice_313.png (completely black).
Deleted sub-strokecase0004_ses-0001_FLAIR.nii_axial_slice_8.png (completely black).
Deleted sub-strokecase0051_ses-0001_FLAIR.nii_axial_slice_43.png (completely black).
Deleted sub-strokecase0090_ses-0001_FLAIR.nii_axial_slice_96.png (completely black).
Deleted sub-strokecase0158_ses-0001_FLAIR.nii_axial_slice_88.png (completely black).
Deleted sub-strokecase0043_ses-0001_FLAIR.nii_axial_slice_311.png (completely black).
Deleted sub-strokecase0018_ses-0001_FLAIR.nii_axial_slice_110

In [16]:
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap

def create_lesion_overlay_images(mri_dir, lesion_dir, output_dir):
    # Create the output directory if it doesn't exist
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    # Define a custom color map with red for the lesions
    lesion_cmap = ListedColormap(['black', 'red'])

    for root, _, files in os.walk(lesion_dir):
        for lesion_file in files:
            if 'msk' in lesion_file:
                lesion_file_path = os.path.join(root, lesion_file)

                # Extract the case number from the lesion file name
                case_number = os.path.basename(lesion_file).split("_")[0][14:]

                # Find the corresponding MRI file
                mri_file = os.path.join(mri_dir, f"sub-strokecase{case_number}", "ses-0001", "dwi", f"sub-strokecase{case_number}_ses-0001_dwi.nii.gz")
                mri_file = glob.glob(mri_file)

                # Check if an MRI file is found for the current case
                if len(mri_file) == 1:
                    mri_file = mri_file[0]
                else:
                    print(f"No MRI file found for case {case_number}. Skipping...")
                    continue

                # Load the MRI and lesion mask data
                mri_img = nib.load(mri_file)
                mri_data = mri_img.get_fdata()
                
#                 with open("./log.log", "w") as f:
#                     f.write(mri_data.shape)
                print(mri_data.shape)
                print(mri_data.shape)
                
                return
                
                lesion_img = nib.load(lesion_file_path)
                lesion_data = lesion_img.get_fdata()

                # Check that both data have the same shape
                if mri_data.shape != lesion_data.shape:
                    print(f"Skipping case {case_number} due to mismatched data shapes.")
                    continue
                    

               
                # Normalize the MRI data to have values between 0 and 1
                mri_data_normalized = (mri_data - mri_data.min()) / (mri_data.max() - mri_data.min())
                
                # Normalize the lesion data to have values between 0 and 1
                lesion_data_normalized = (lesion_data - lesion_data.min()) / (lesion_data.max() - lesion_data.min())

                # Create the lesion overlay images for each slice of the axial view
                for i in range(mri_data.shape[2]):
                    if np.any(lesion_data_normalized[:, :, i] > 0):
                        overlay_image = np.zeros_like(mri_data_normalized)
#                         output_file = os.path.join("./data/lesion-normal-vis", f"sub-strokecase{case_number}", f"case{case_number}_{i}_normal.png")
#                         os.makedirs(os.path.dirname(output_file), exist_ok=True)
                        plt.imshow(mri_data_normalized[:, :, i], cmap='gray', alpha=1.0)  # Display MRI in gray scale with full opacity
#                         plt.savefig(output_file, bbox_inches='tight', pad_inches=0, dpi=100)

                        # Combine the MRI and lesion data in the overlay image
                        overlay_image[:, :, i] = mri_data_normalized[:, :, i]
                        overlay_image[:, :, i][lesion_data_normalized[:, :, i] > 0] = 1.0

                        # Create and save the PNG image with the lesions colored red
                        output_file = os.path.join(output_dir, f"sub-strokecase{case_number}", f"case{case_number}_lesion_overlay_slice_{i}.png")
                        print(output_file)
                        os.makedirs(os.path.dirname(output_file), exist_ok=True)  # Create case sub-directory if it doesn't exist
#                         plt.imshow(overlay_image[:, :, i], cmap='gray', alpha=1.0)  # Display MRI in gray scale with full opacity
                        plt.imshow(lesion_data_normalized[:, :, i], cmap=lesion_cmap, alpha=0.15)  # Display lesions with partial opacity
                        plt.axis('off')
                        plt.savefig(output_file, bbox_inches='tight', pad_inches=0, dpi=100)
                        plt.close()

In [17]:
input_lesion_mask_dir = "./data/unused/ISLES-2022/derivatives"
input_mri_dir = "./data/unused/ISLES-2022"
output_mask_dir = "./data/lesion_mask"
create_lesion_overlay_images(input_mri_dir, input_lesion_mask_dir, output_mask_dir)

(192, 192, 30)
(192, 192, 30)


In [23]:

def convert_slices_with_mask_to_png(input_dir, mask_dir, output_dir):
    # Create the output directory if it doesn't exist
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    for root, _, files in os.walk(mask_dir):
        for mask_file in files:
            if 'lesion_overlay_slice' in mask_file:
                mask_file_path = os.path.join(root, mask_file)

                # Extract the case number and slice number from the mask file name
                case_number = os.path.basename(mask_file).split("_")[0][4:]
                slice_number = int(mask_file.split("_")[-1].split(".")[0])

                # Find the corresponding DWI NIfTI file
                dwi_file = os.path.join(input_dir, f"sub-strokecase{case_number}", "ses-0001", "dwi", f"*dwi*")
                dwi_file = glob.glob(dwi_file)

                # Check if a DWI NIfTI file is found for the current case
                if len(dwi_file) != 1:
                    print(f"No DWI NIfTI file found for case {case_number}. Skipping...")
                    continue

                # Load the DWI data
                dwi_img = nib.load(dwi_file[0])
                dwi_data = dwi_img.get_fdata()

                # Retrieve the corresponding slice from the DWI data
                dwi_slice = dwi_data[:, :, slice_number]

                # Normalize the DWI slice to have values between 0 and 1 for better visualization
                dwi_slice_normalized = (dwi_slice - dwi_slice.min()) / (dwi_slice.max() - dwi_slice.min())

                # Create and save the PNG image
                output_file = os.path.join(output_dir, f"sub-strokecase{case_number}", f"case{case_number}_dwi_slice_{slice_number}.png")
                os.makedirs(os.path.dirname(output_file), exist_ok=True)  # Create case sub-directory if it doesn't exist
                plt.imshow(dwi_slice_normalized, cmap='gray')
                plt.axis('off')
                plt.savefig(output_file, bbox_inches='tight', pad_inches=0, dpi=100)
                plt.close()

                print(f"Processed case {case_number}, slice {slice_number}, and saved the DWI slice as PNG to {os.path.join(output_dir, f'sub-strokecase{case_number}')}.")

In [26]:
input_directory = "./data/ISLES-2022"
output_directory = "./data/lesion_images"
mask_directory = "./data/lesion_mask"
convert_slices_with_mask_to_png(input_directory, mask_directory, output_directory)

Processed case 0248, slice 13, and saved the DWI slice as PNG to ./data/lesion_images/sub-strokecase0248.
Processed case 0248, slice 14, and saved the DWI slice as PNG to ./data/lesion_images/sub-strokecase0248.
Processed case 0248, slice 19, and saved the DWI slice as PNG to ./data/lesion_images/sub-strokecase0248.
Processed case 0248, slice 25, and saved the DWI slice as PNG to ./data/lesion_images/sub-strokecase0248.
Processed case 0248, slice 9, and saved the DWI slice as PNG to ./data/lesion_images/sub-strokecase0248.
Processed case 0248, slice 23, and saved the DWI slice as PNG to ./data/lesion_images/sub-strokecase0248.
Processed case 0248, slice 18, and saved the DWI slice as PNG to ./data/lesion_images/sub-strokecase0248.
Processed case 0248, slice 16, and saved the DWI slice as PNG to ./data/lesion_images/sub-strokecase0248.
Processed case 0248, slice 24, and saved the DWI slice as PNG to ./data/lesion_images/sub-strokecase0248.
Processed case 0248, slice 21, and saved the DW

Processed case 0089, slice 41, and saved the DWI slice as PNG to ./data/lesion_images/sub-strokecase0089.
Processed case 0089, slice 39, and saved the DWI slice as PNG to ./data/lesion_images/sub-strokecase0089.
Processed case 0089, slice 40, and saved the DWI slice as PNG to ./data/lesion_images/sub-strokecase0089.
Processed case 0089, slice 37, and saved the DWI slice as PNG to ./data/lesion_images/sub-strokecase0089.
No DWI NIfTI file found for case 0177. Skipping...
No DWI NIfTI file found for case 0177. Skipping...
No DWI NIfTI file found for case 0177. Skipping...
No DWI NIfTI file found for case 0177. Skipping...
No DWI NIfTI file found for case 0177. Skipping...
No DWI NIfTI file found for case 0177. Skipping...
No DWI NIfTI file found for case 0211. Skipping...
No DWI NIfTI file found for case 0211. Skipping...
No DWI NIfTI file found for case 0211. Skipping...
No DWI NIfTI file found for case 0211. Skipping...
No DWI NIfTI file found for case 0211. Skipping...
No DWI NIfTI fi

Processed case 0236, slice 11, and saved the DWI slice as PNG to ./data/lesion_images/sub-strokecase0236.
Processed case 0236, slice 13, and saved the DWI slice as PNG to ./data/lesion_images/sub-strokecase0236.
Processed case 0236, slice 3, and saved the DWI slice as PNG to ./data/lesion_images/sub-strokecase0236.
Processed case 0236, slice 12, and saved the DWI slice as PNG to ./data/lesion_images/sub-strokecase0236.
Processed case 0236, slice 19, and saved the DWI slice as PNG to ./data/lesion_images/sub-strokecase0236.
Processed case 0236, slice 22, and saved the DWI slice as PNG to ./data/lesion_images/sub-strokecase0236.
Processed case 0236, slice 17, and saved the DWI slice as PNG to ./data/lesion_images/sub-strokecase0236.
Processed case 0236, slice 14, and saved the DWI slice as PNG to ./data/lesion_images/sub-strokecase0236.
Processed case 0236, slice 7, and saved the DWI slice as PNG to ./data/lesion_images/sub-strokecase0236.
Processed case 0236, slice 20, and saved the DWI

Processed case 0054, slice 37, and saved the DWI slice as PNG to ./data/lesion_images/sub-strokecase0054.
Processed case 0054, slice 23, and saved the DWI slice as PNG to ./data/lesion_images/sub-strokecase0054.
Processed case 0054, slice 41, and saved the DWI slice as PNG to ./data/lesion_images/sub-strokecase0054.
Processed case 0054, slice 45, and saved the DWI slice as PNG to ./data/lesion_images/sub-strokecase0054.
Processed case 0054, slice 36, and saved the DWI slice as PNG to ./data/lesion_images/sub-strokecase0054.
Processed case 0054, slice 35, and saved the DWI slice as PNG to ./data/lesion_images/sub-strokecase0054.
Processed case 0054, slice 33, and saved the DWI slice as PNG to ./data/lesion_images/sub-strokecase0054.
Processed case 0054, slice 20, and saved the DWI slice as PNG to ./data/lesion_images/sub-strokecase0054.
Processed case 0054, slice 34, and saved the DWI slice as PNG to ./data/lesion_images/sub-strokecase0054.
Processed case 0054, slice 43, and saved the D

Processed case 0175, slice 12, and saved the DWI slice as PNG to ./data/lesion_images/sub-strokecase0175.
Processed case 0175, slice 15, and saved the DWI slice as PNG to ./data/lesion_images/sub-strokecase0175.
Processed case 0043, slice 31, and saved the DWI slice as PNG to ./data/lesion_images/sub-strokecase0043.
Processed case 0043, slice 45, and saved the DWI slice as PNG to ./data/lesion_images/sub-strokecase0043.
Processed case 0043, slice 36, and saved the DWI slice as PNG to ./data/lesion_images/sub-strokecase0043.
Processed case 0043, slice 12, and saved the DWI slice as PNG to ./data/lesion_images/sub-strokecase0043.
Processed case 0043, slice 19, and saved the DWI slice as PNG to ./data/lesion_images/sub-strokecase0043.
Processed case 0043, slice 61, and saved the DWI slice as PNG to ./data/lesion_images/sub-strokecase0043.
Processed case 0043, slice 38, and saved the DWI slice as PNG to ./data/lesion_images/sub-strokecase0043.
Processed case 0043, slice 54, and saved the D

...

Processed case 0230, slice 12, and saved the DWI slice as PNG to ./data/lesion_images/sub-strokecase0230.
Processed case 0230, slice 10, and saved the DWI slice as PNG to ./data/lesion_images/sub-strokecase0230.
Processed case 0230, slice 21, and saved the DWI slice as PNG to ./data/lesion_images/sub-strokecase0230.
Processed case 0230, slice 18, and saved the DWI slice as PNG to ./data/lesion_images/sub-strokecase0230.
Processed case 0230, slice 19, and saved the DWI slice as PNG to ./data/lesion_images/sub-strokecase0230.
Processed case 0230, slice 14, and saved the DWI slice as PNG to ./data/lesion_images/sub-strokecase0230.
Processed case 0230, slice 8, and saved the DWI slice as PNG to ./data/lesion_images/sub-strokecase0230.
Processed case 0230, slice 15, and saved the DWI slice as PNG to ./data/lesion_images/sub-strokecase0230.
Processed case 0230, slice 22, and saved the DWI slice as PNG to ./data/lesion_images/sub-strokecase0230.
Processed case 0138, slice 16, and saved the DW

No DWI NIfTI file found for case 0224. Skipping...
No DWI NIfTI file found for case 0224. Skipping...
No DWI NIfTI file found for case 0224. Skipping...
No DWI NIfTI file found for case 0224. Skipping...
No DWI NIfTI file found for case 0224. Skipping...
No DWI NIfTI file found for case 0224. Skipping...
No DWI NIfTI file found for case 0224. Skipping...
No DWI NIfTI file found for case 0224. Skipping...
No DWI NIfTI file found for case 0224. Skipping...
No DWI NIfTI file found for case 0224. Skipping...
No DWI NIfTI file found for case 0224. Skipping...
No DWI NIfTI file found for case 0224. Skipping...
No DWI NIfTI file found for case 0224. Skipping...
No DWI NIfTI file found for case 0224. Skipping...
No DWI NIfTI file found for case 0224. Skipping...
No DWI NIfTI file found for case 0224. Skipping...
No DWI NIfTI file found for case 0224. Skipping...
No DWI NIfTI file found for case 0224. Skipping...
No DWI NIfTI file found for case 0224. Skipping...
No DWI NIfTI file found for cas

Processed case 0066, slice 43, and saved the DWI slice as PNG to ./data/lesion_images/sub-strokecase0066.
Processed case 0066, slice 46, and saved the DWI slice as PNG to ./data/lesion_images/sub-strokecase0066.
Processed case 0066, slice 30, and saved the DWI slice as PNG to ./data/lesion_images/sub-strokecase0066.
Processed case 0066, slice 39, and saved the DWI slice as PNG to ./data/lesion_images/sub-strokecase0066.
Processed case 0066, slice 29, and saved the DWI slice as PNG to ./data/lesion_images/sub-strokecase0066.
Processed case 0066, slice 47, and saved the DWI slice as PNG to ./data/lesion_images/sub-strokecase0066.
Processed case 0066, slice 41, and saved the DWI slice as PNG to ./data/lesion_images/sub-strokecase0066.
Processed case 0066, slice 37, and saved the DWI slice as PNG to ./data/lesion_images/sub-strokecase0066.
Processed case 0066, slice 42, and saved the DWI slice as PNG to ./data/lesion_images/sub-strokecase0066.
Processed case 0066, slice 21, and saved the D

Processed case 0119, slice 50, and saved the DWI slice as PNG to ./data/lesion_images/sub-strokecase0119.
Processed case 0119, slice 47, and saved the DWI slice as PNG to ./data/lesion_images/sub-strokecase0119.
No DWI NIfTI file found for case 0158. Skipping...
No DWI NIfTI file found for case 0158. Skipping...
No DWI NIfTI file found for case 0158. Skipping...
No DWI NIfTI file found for case 0158. Skipping...
No DWI NIfTI file found for case 0158. Skipping...
No DWI NIfTI file found for case 0158. Skipping...
No DWI NIfTI file found for case 0158. Skipping...
No DWI NIfTI file found for case 0158. Skipping...
No DWI NIfTI file found for case 0158. Skipping...
No DWI NIfTI file found for case 0158. Skipping...
No DWI NIfTI file found for case 0158. Skipping...
No DWI NIfTI file found for case 0158. Skipping...
No DWI NIfTI file found for case 0158. Skipping...
No DWI NIfTI file found for case 0158. Skipping...
No DWI NIfTI file found for case 0158. Skipping...
No DWI NIfTI file found

Processed case 0186, slice 9, and saved the DWI slice as PNG to ./data/lesion_images/sub-strokecase0186.
Processed case 0186, slice 20, and saved the DWI slice as PNG to ./data/lesion_images/sub-strokecase0186.
Processed case 0186, slice 22, and saved the DWI slice as PNG to ./data/lesion_images/sub-strokecase0186.
Processed case 0186, slice 16, and saved the DWI slice as PNG to ./data/lesion_images/sub-strokecase0186.
Processed case 0186, slice 19, and saved the DWI slice as PNG to ./data/lesion_images/sub-strokecase0186.
Processed case 0186, slice 17, and saved the DWI slice as PNG to ./data/lesion_images/sub-strokecase0186.
Processed case 0186, slice 8, and saved the DWI slice as PNG to ./data/lesion_images/sub-strokecase0186.
Processed case 0186, slice 11, and saved the DWI slice as PNG to ./data/lesion_images/sub-strokecase0186.
Processed case 0186, slice 7, and saved the DWI slice as PNG to ./data/lesion_images/sub-strokecase0186.
Processed case 0186, slice 5, and saved the DWI s

### Create metadata.jsonl

In [98]:
import json

prompts = {
    "h2h": "This is an image of a normal brain MRI scan from axial view. It should be grey scale and clearly show brain structure. Do not modify the input image.",
    "h2uh": "Given an input image of a normal brain MRI scan from axial view, modify the image by inserting a brain lesion within the MRI image. This brain lesion is caused by an ischaemic stroke. The lesion should appear relatively lighter compared to its surroundings brain tissue in the image.",
    "uh2uhm": "Given an input image of an unhealthy brain MRI scan from axial view that contains brain lesions, modify the image to colour the lesions light red.",
    "h2uhm": "Given an input image of a normal brain MRI scan from axial view, modify the image by inserting a brain lesion within the MRI image, and colour the lesions light red. This brain lesion is caused by an ischaemic stroke.",
    "uhm2uhm": "This is an image of an unhealthy brain MRI scan from axial view that contains brain lesions. The lesions are coloured light red."
}

healthy_data_dir = "./data/healthy_images"
lesion_data_dir = "./data/lesion_images"
lesion_mask_data_dir = "./data/lesion_mask"

def get_images(data_dir, max_dir=10, img_range=[30,60], lesion_mask=False):
    result = []
    image_directories = os.listdir(data_dir)[:max_dir]
    for _dir in image_directories:
        img_files = os.listdir(os.path.join(data_dir, _dir))
        for file in img_files:
            slice_number_and_extension = os.path.basename(file).split("_")[-1]
            slice_number, file_extension = os.path.splitext(slice_number_and_extension)
            slice_number = int(slice_number)
            if slice_number < img_range[0] or slice_number > img_range[1]:
                continue
            result.append(os.path.join(data_dir.split("/")[-1], _dir, file))
    return result

def get_lesion_images(data_dir, mask_data_dir, max_img=20000, img_range=[30,60]):
    result = []
    image_directories = os.listdir(data_dir)
    for _dir in image_directories:
        img_files = os.listdir(os.path.join(data_dir, _dir))
        for file in img_files:
            # Check if mask exists
            respective_mask_path = os.path.join(mask_data_dir, _dir, file.replace("dwi", "lesion_overlay"))
            if not os.path.exists(respective_mask_path):
                print("skipping")
                continue
            slice_number_and_extension = os.path.basename(file).split("_")[-1]
            slice_number, file_extension = os.path.splitext(slice_number_and_extension)
            slice_number = int(slice_number)
            if slice_number < img_range[0] or slice_number > img_range[1]:
                continue
            result.append(os.path.join(data_dir.split("/")[-1], _dir, file))
            if len(result) >= max_img:
                return result
    return result

healthy_images = get_images(healthy_data_dir)
lesion_images = get_lesion_images(lesion_data_dir, lesion_mask_data_dir, img_range=[30, 50])
print(len(healthy_images))
print(len(lesion_images))

310
310


In [99]:
def do_train_split(healthy_img_list, lesion_img_list):
    data = []
    for idx, healthy_img in enumerate(healthy_img_list):
        file_name = healthy_img
        input_image = {"src":healthy_img, "height":301,"width":302}
        edit_prompt = prompts["h2h"]
        edited_image = input_image
        _object = {
            "file_name": file_name,
            "input_image": input_image,
            "edit_prompt": edit_prompt,
            "edited_image": edited_image
        }
        data.append(_object)
    
    for idx, healthy_img in enumerate(healthy_img_list):
        file_name = healthy_img
        input_image = {"src":healthy_img, "height":301,"width":302}
        edit_prompt = prompts["h2uh"]
        edited_image = {"src":lesion_img_list[idx], "height":301,"width":302}
        _object = {
            "file_name": file_name,
            "input_image": input_image,
            "edit_prompt": edit_prompt,
            "edited_image": edited_image
        }
        data.append(_object)
        
    for idx, healthy_img in enumerate(healthy_img_list):
        file_name = healthy_img
        input_image = {"src":healthy_img, "height":301,"width":302}
        edit_prompt = prompts["h2uhm"]
        lesion_mask_path = lesion_img_list[idx].replace("images", "mask").replace("dwi", "lesion_overlay")
        edited_image = {"src": lesion_mask_path, "height":301,"width":302}
        _object = {
            "file_name": file_name,
            "input_image": input_image,
            "edit_prompt": edit_prompt,
            "edited_image": edited_image
        }
        data.append(_object)
            
    for idx, lesion_img in enumerate(lesion_img_list):
        file_name = lesion_img
        input_image = {"src":lesion_img, "height":301,"width":302}
        edit_prompt = prompts["uh2uhm"]
        lesion_mask_path = lesion_img.replace("images", "mask").replace("dwi", "lesion_overlay")
        edited_image = {"src": lesion_mask_path, "height":301,"width":302}
        _object = {
            "file_name": file_name,
            "input_image": input_image,
            "edit_prompt": edit_prompt,
            "edited_image": edited_image
        }
        data.append(_object)
        
        lesion_mask_path = lesion_img.replace("images", "mask").replace("dwi", "lesion_overlay")
        file_name = lesion_mask_path
        input_image = {"src":lesion_mask_path, "height":301,"width":302}
        edit_prompt = prompts["uhm2uhm"]
        edited_image = {"src": lesion_mask_path, "height":301,"width":302}
        _object = {
            "file_name": file_name,
            "input_image": input_image,
            "edit_prompt": edit_prompt,
            "edited_image": edited_image
        }
        data.append(_object)
        
    return data

data = do_train_split(healthy_images, lesion_images)



In [100]:
# File path for the JSONL file
file_path = "./data/train/metadata.jsonl"

# Writing JSON objects to the file
with open(file_path, "w") as jsonl_file:
    for item in data:
        jsonl_file.write(json.dumps(item) + "\n")

In [101]:
import shutil
train_data_dir = "./data/train"

os.makedirs(os.path.join(train_data_dir, "healthy_images"), exist_ok=True)
os.makedirs(os.path.join(train_data_dir, "lesion_images"), exist_ok=True)
os.makedirs(os.path.join(train_data_dir, "lesion_mask"), exist_ok=True)

def copy_file(source_file, destination_file):
    try:
        # Copy the file from source to destination
        shutil.copy(source_file, destination_file)
        print(f"File copied successfully from '{source_file}' to '{destination_file}'.")
    except FileNotFoundError:
        print("The source file does not exist.")
    except PermissionError:
        print("Permission denied. You may not have the required permissions to copy the file.")
    except Exception as e:
        print(f"An error occurred: {e}")

# # Copy relevant files over
for item in data:
    filenames = [item["file_name"], item["input_image"]["src"], item["edited_image"]["src"]]
    for filename in filenames:
        train_img_path = os.path.join(train_data_dir, filename)
        if os.path.exists(train_img_path):
            continue

        directories = filename.split(os.sep)
        print(directories)

        ori_img_path = os.path.join("./data/", filename)
        os.makedirs(os.path.join(train_data_dir, directories[0], directories[1]), exist_ok=True)
        copy_file(ori_img_path, train_img_path)    

['healthy_images', 'sub-ON54268', 'caseON54268_dwi_slice_54.png']
File copied successfully from './data/healthy_images/sub-ON54268/caseON54268_dwi_slice_54.png' to './data/train/healthy_images/sub-ON54268/caseON54268_dwi_slice_54.png'.
['healthy_images', 'sub-ON54268', 'caseON54268_dwi_slice_37.png']
File copied successfully from './data/healthy_images/sub-ON54268/caseON54268_dwi_slice_37.png' to './data/train/healthy_images/sub-ON54268/caseON54268_dwi_slice_37.png'.
['healthy_images', 'sub-ON54268', 'caseON54268_dwi_slice_59.png']
File copied successfully from './data/healthy_images/sub-ON54268/caseON54268_dwi_slice_59.png' to './data/train/healthy_images/sub-ON54268/caseON54268_dwi_slice_59.png'.
['healthy_images', 'sub-ON54268', 'caseON54268_dwi_slice_42.png']
File copied successfully from './data/healthy_images/sub-ON54268/caseON54268_dwi_slice_42.png' to './data/train/healthy_images/sub-ON54268/caseON54268_dwi_slice_42.png'.
['healthy_images', 'sub-ON54268', 'caseON54268_dwi_slice